In [2]:
__author__ = '@iRodGit'
import numpy as np
import pandas as pd
import os
from PIL import Image
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Add
from keras.layers import AveragePooling2D, Conv2D, MaxPooling2D, concatenate # Concatenate
from keras.models import Sequential, Model
from keras.utils import to_categorical
from codvidutils.imageproc import map_categorical

In [3]:

# example of loading the inception v3 model
from keras.applications.inception_v3 import InceptionV3
# load model
model = InceptionV3(include_top=False, input_shape =(180,180,3))
# summarize the model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 180, 180, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 89, 89, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 89, 89, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 89, 89, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
#conv = Conv2D(filters= 4096, kernel_size = (7,7), activation = "relu")(modeloutputs)
flat1 = Flatten()(model.outputs)
class1 = Dense(128, activation='relu')(flat1)
output = Dense(1, activation='sigmoid')(class1)
# define new model
prob = Model(inputs=model.inputs, outputs=output)
prob.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 180, 180, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 89, 89, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 89, 89, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 89, 89, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
conv_FC = Conv2D(filters =128,kernel_size =(1,1),activation="relu")(model.outputs)
first_dense = Conv2D(filters =32,kernel_size =(1,1),activation="relu")(conv_FC)
#sec_dense = Conv2D(filters =16,kernel_size =(1,1),activation="relu")(first_dense)
flat = Flatten()(first_dense)
out_layer= Dense(units =1, activation= "sigmoid")(flat)
deep = Model(inputs=model.inputs, outputs=out_layer)

In [12]:
train_class = pd.read_csv('data/train_split_v2.txt', sep=' ', header=None, names=['patientID', 'image_path', 'class'])
test_class = pd.read_csv('data/test_split_v2.txt', sep=' ', header=None, names=['patientID', 'image_path', 'class'])

In [14]:
pics = []
for img in train_class['image_path'].values:
    pics.append(np.array(Image.open('data/train/' + img))[:, :,:3])
#test_pics = []
for img in test_class['image_path'].values:
    pics.append(np.array(Image.open('data/test/' + img))[:, :, :3])

print("Number of images:", len(pics))

Number of images: 18527


In [15]:
values_dict = {'COVID-19': 1,'pneumonia': 2,'normal': 0}
test_class['class_categorical'] = test_class['class'].apply(map_categorical, args=(values_dict,))
train_class['class_categorical'] = train_class['class'].apply(map_categorical, args=(values_dict,))

In [16]:
X = np.array(pics)
del pics
diseaseID_train = np.asarray(train_class["class_categorical"])
diseaseID_test = np.asarray(test_class["class_categorical"])
diseaseID = np.concatenate([diseaseID_train, diseaseID_test],axis=0)
print('shape X: {},  Y: {}'.format(X.shape, diseaseID.shape ))

shape X: (18527, 200, 200, 3),  Y: (18527,)


In [17]:
from collections import Counter
counter = Counter (diseaseID)
print(counter)

Counter({2: 9579, 0: 8851, 1: 97})


In [21]:
X.shape

(18527, 200, 200, 3)

In [22]:
dicto = {2: 4500, 0: 4500, 1:97}
print(dicto)
from imblearn.under_sampling import RandomUnderSampler
#X = X.reshape(X,X.shape[0],-1)
X = X.reshape(X.shape[0],-1)
print(X.shape)
under = RandomUnderSampler(sampling_strategy =dicto)
X, diseaseID = under.fit_resample(X, diseaseID)
# summarize class distribution
print(diseaseID.shape)
print(X.shape)

{0: 4500, 1: 97, 2: 4500}
(18527, 120000)
(9097,)
(9097, 120000)


In [23]:
n = np.random.randint(1000,6760)
print(n)
from sklearn.utils import shuffle
X, diseaseID = shuffle(X, diseaseID, random_state=n)
from sklearn.model_selection import train_test_split
m = np.random.randint(1000,6760)
print(m)
X_train, X_test, diseaseID_train, diseaseID_test = train_test_split(X, diseaseID, test_size=0.20, random_state=m)
del X, diseaseID

3000
3961


In [26]:
print("Normal train: ",diseaseID_train[diseaseID_train==0].shape)
print("Pneumonia train: ",diseaseID_train[diseaseID_train==2].shape)
print("COVID train: ",diseaseID_train[diseaseID_train==1].shape)
print("*******************************************************")
print("Normal test: ",diseaseID_test[diseaseID_test==0].shape)
print("Pneumonia test: ",diseaseID_test[diseaseID_test==2].shape)
print("COVID test: ",diseaseID_test[diseaseID_test==1].shape)

Normal train:  (3615,)
Pneumonia train:  (3581,)
COVID train:  (81,)
*******************************************************
Normal test:  (885,)
Pneumonia test:  (919,)
COVID test:  (16,)


In [28]:
def new_pictures_arrays(pictures, strides=5, kernel=(180,180)):
    news = []
    for i in range(int((pictures.shape[1]-kernel[0])/strides)+1):
        for j in range(int((pictures.shape[2]-kernel[1])/strides)+1):
            s,k = strides*i, strides*j
            news.append(pictures[:,s:kernel[0]+s,k:kernel[1]+k])
    news= np.array(news)
    
    return news.reshape(news.shape[0]*news.shape[1], kernel[0],kernel[1],pictures.shape[3])

In [29]:
"""
News images to train 
"""
X_train_news = new_pictures_arrays(X_train[diseaseID_train==1])
print(X_train_news.shape)
diseaseID_train_news = np.ones(X_train_news.shape[0])
print(diseaseID_train_news.shape)
X_train = X_train[:,10:190,10:190]
X_train = np.concatenate([X_train,X_train_news],axis=0)
diseaseID_train = np.concatenate([diseaseID_train,diseaseID_train_news],axis=0)
del X_train_news, diseaseID_train_news
print(X_train.shape)
print(diseaseID_train.shape)

(2025, 180, 180, 3)
(2025,)
(9302, 180, 180, 3)
(9302,)


In [30]:
"""
News images to test 
"""
X_test_news = new_pictures_arrays(X_test[diseaseID_test==1])
print(X_test_news.shape)
diseaseID_test_news = np.ones(X_test_news.shape[0])
print(diseaseID_test_news.shape)
X_test = X_test[:,10:190,10:190]
X_test = np.concatenate([X_test,X_test_news],axis=0)
diseaseID_test = np.concatenate([diseaseID_test,diseaseID_test_news],axis=0)
del X_test_news, diseaseID_test_news
print(X_test.shape)
print(diseaseID_test.shape)

(400, 180, 180, 3)
(400,)
(2220, 180, 180, 3)
(2220,)


In [32]:
X_test = X_test/255
X_train = X_train/255

In [33]:
Y_train = np.copy(diseaseID_train)
del diseaseID_train
Y_train[Y_train==2]=0
Y_test = np.copy(diseaseID_test)
Y_test[Y_test==2]=0

X_test = X_test/255
X_train = X_train/255

In [34]:
print('Train shape X: {},  Y: {} \n Test shape X: {}, Y: {}'.format(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape))

Train shape X: (9302, 180, 180, 3),  Y: (9302,) 
 Test shape X: (2220, 180, 180, 3), Y: (2220,)


In [35]:
"""
Weighted loss
"""
covid_count, no_covid_count = Y_train[Y_train==1].shape[0],Y_train[Y_train==0].shape[0]
total_count = covid_count + no_covid_count
print(covid_count, no_covid_count)
weight_no_covid = covid_count / total_count
weight_covid = no_covid_count / total_count
dic_weights = {0: weight_no_covid, 1: weight_covid}
print(dic_weights)

2106 7196
{0: 0.22640292410234358, 1: 0.7735970758976565}


In [39]:
config = tf.ConfigProto(device_count={'GPU': 1, 'CPU': 3})
sess = tf.Session(config= config)
keras.backend.set_session(sess)

In [ ]:
from keras.callbacks import ModelCheckpoint
ASG = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=True)
filepath="Inception_bestmodel_trained.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

prob.compile(optimizer=ASG, loss='binary_crossentropy', metrics=['acc',"mse"])
history = prob.fit(X_train, Y_train, epochs=12, batch_size=32, callbacks= [checkpoint],verbose=2, class_weight=dic_weights,validation_data=(X_test, Y_test))


Train on 9302 samples, validate on 2220 samples
Epoch 1/12


In [41]:
from keras.utils import plot_model
plot_model(prob, to_file='probando.png')
